In [64]:
import noise
import numpy as np
from PIL import Image, ImageEnhance


for x in range(15000):
    shape = (512,512)
    scale = .5
    octaves = 6
    persistence = 0.5
    lacunarity = 2.0
    seed = x
    world = np.zeros(shape)
    # make coordinate grid on [0,1]^2
    x_idx = np.linspace(0, 1, shape[0])
    y_idx = np.linspace(0, 1, shape[1])
    world_x, world_y = np.meshgrid(x_idx, y_idx)
    # apply perlin noise, instead of np.vectorize, consider using itertools.starmap()
    world = np.vectorize(noise.snoise2)(world_x/scale,
                            world_y/scale,
                            octaves=octaves,
                            persistence=persistence,
                            lacunarity=lacunarity,
                            repeatx=512,
                            repeaty=512,
                            base=seed)
    # here was the error: one needs to normalize the image first. Could be done without copying the array, though
    hieghtmap = np.round((world + .555) * 200).astype(np.uint8) # <- Normalize world first
    img = Image.fromarray(hieghtmap, mode='L')
    img.save(str(x)+".png")
    
    img2 = Image.open(str(x)+".png")
    img2 = img2.convert('RGBA')

    data = np.array(img2)   # "data" is a height x width x 4 numpy array
    red, green, blue, alpha = data.T # Temporarily unpack the bands for readability

    # Replace white with red... (leaves alpha values alone...)
    for whiteAreaRange in range(236,256):
        white_areas = (red == whiteAreaRange) & (blue == whiteAreaRange) & (green == whiteAreaRange)
        data[..., :-1][white_areas.T] = (0, 0, 0) # Transpose back needed
    
    out_image2 = Image.fromarray(data)
    
    enhancer = ImageEnhance.Contrast(out_image2)
    im_output = enhancer.enhance(1.5)
    out_image2.save(str(x)+".png")

